# 6. 如何开展和记录实验

导读: 当我们开始训练多个具有不同超参数的模型，我们就需要对实验开始进行管理。**我们将其分为两个部分：实验追踪、和配置设置**。我们将使用SwanLab来演示实验记录和追踪；然后，学习如何配置我们深度学习应用的参数。

本次课程目的在于能够让你了解并实践如何将实验管理工具整合到你的模型训练工作流程中。本节还是在上一个图像分类任务代码的基础上继续进行改进。

## 本教程目标

1. 通过SwanLab管理实验记录
2. 了解参数配置。

## 本教程内容

0. 训练流程

这是第2节课的代码，如果不熟悉，再回去看视频讲解，多看几遍

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# 1.构建数据集
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 2.定义神经网络
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

# 3.定义 Loss 函数和优化器
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # risk loss
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# 4.训练网络
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        if (i > 5):
          break # 为了增加训练速度，正常需要训练所有数据
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Files already downloaded and verified
Files already downloaded and verified
Finished Training


## 1. 实验记录

大家是不是都曾遇到过这样的情况：如果没有良好的实验记录工具，我们最终也许会得到一个性能非常好的模型，但我们不记得其超参数选择，或者启动 100 个实验却无法轻松跟踪哪个模型表现最好，而实验跟踪工具能帮助我们解决这些问题。

**Logging**

通常来说我们在训练的过程中，通常会打印我们正在使用的超参数，以及模型训练时的损失+准确性。就比如上面打印的结果一般。我们能看到每一个epoch的损失是多少。下面展示我们如何用SwanLab管理实验记录

### SwanLab

SwanLab是一款开源、轻量级的AI实验跟踪工具，提供了一个跟踪、比较、和协作实验的平台，旨在加速AI研发团队100倍的研发效率。

其提供了友好的API和漂亮的界面，结合了超参数跟踪、指标记录、在线协作、实验链接分享、实时消息通知等功能，让您可以快速跟踪ML实验、可视化过程、分享给同伴。

借助SwanLab，科研人员可以沉淀自己的每一次训练经验，与合作者无缝地交流和协作，机器学习工程师可以更快地开发可用于生产的模型。

In [ ]:
pip install swanlab

In [ ]:
import swanlab
swanlab.login()

下面重新构造上面的训练流程，以便我们能够记录参数

In [ ]:
def train(epochs, learning_rate):
  print(f"Training for {epochs} epochs with learning rate {learning_rate}")

  swanlab.init(
        # Set the project where this run will be logged
        project="example",
        # Track hyperparameters and run metadata
        config={
        "learning_rate": learning_rate,
        "epochs": epochs,
        })


  # 构造数据集
  transform = transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  batch_size = 4

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,

                                            shuffle=True, num_workers=2)
  # 定义网络
  net = Net()

  # 定义损失和优化器
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9) # 学习率作为一个可以调整的参数


  # 训练网络
  for epoch in range(epochs):  # epochs作为参数传入

      running_loss = 0.0
      for i, data in enumerate(trainloader, 0):
          # get the inputs; data is a list of [inputs, labels]
          if (i > 5):
            break
          inputs, labels = data

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
      print(f"epoch={epoch}, loss={running_loss}")
      swanlab.log({"loss": running_loss})
      running_loss = 0.0
  swanlab.finish()


In [ ]:
train(epochs=10, learning_rate=0.01)

Training for 10 epochs with learning rate 0.01
swanlab: Tracking run with swanlab version 0.3.23
swanlab: Run data will be saved locally in /content/swanlog/run-20241027_170236-11c31ea9
swanlab: 👋 Hi wenglean, welcome to swanlab!
swanlab: Syncing run snake-7 to the cloud
swanlab: 🌟 Run `swanlab watch /content/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@wenglean/example
swanlab: 🚀 View run at https://swanlab.cn/@wenglean/example/runs/fxz59ri9m3g7jxbm46owh


Files already downloaded and verified
epoch=0, loss=13.845096826553345
epoch=1, loss=13.9386727809906
epoch=2, loss=13.875343084335327
epoch=3, loss=13.757331609725952
epoch=4, loss=13.789016962051392
epoch=5, loss=13.851541519165039
epoch=6, loss=13.88466501235962
epoch=7, loss=13.675924301147461
epoch=8, loss=13.8859281539917
epoch=9, loss=13.772759199142456
swanlab: 🌟 Run `swanlab watch /content/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@wenglean/example
swanlab: 🚀 View run at https://swanlab.cn/@wenglean/example/runs/fxz59ri9m3g7jxbm46owh


我们在这里使用 3 个函数：swanlab.init、swanlab.log 和 swanlab.finish——它们各自的作用是什么？

- 我们在脚本开头调用一次 swanlab.init() 来初始化新项目。这会创建新的运行并启动后台进程来同步数据。
- 我们调用 swanlab.log(dict) 将指标、媒体或自定义对象的字典记录到步骤中。我们可以看到我们的模型和数据如何随着时间的推移而演变。
- 我们调用swanlab.finish来使运行完成，并完成所有数据的上传。

让我们看看在 swanlab 网站上看到了什么，应该看到我们的准确性和损失曲线。

我们已经获得了两个不错的功能：

1. 能够看到循环每一步的准确性和损失如何变化。
2. 能够看到与运行相关的配置（超参数）。
3. 能够看到我们的运行最终获得的准确率acc和loss损失。

# 参数进行配置

我们不希望用硬编码的路径名、模型名和超参数来训练深度学习模型。我们希望能够使用一个配置文件，根据使用的数据集、模型或配置进行修改。硬编码是什么？是指在编写程序时，直接将具体的值（如字符串、数字、路径等）写入源代码中，而不是通过变量、配置文件、数据库查询或其他动态方法来获取这些值。(这其实不是一个好习惯，但是经常有人这样做)

首先，让我们从一些错误的配置深度学习运行的方法开始。假设我们想从命令行控制数据集的 batch_size。可能在某台机器上工作时，你可以使用较大的 batch_size，而在另一台机器上则不行。最基本的做法是记住更改硬编码的 batch size。

In [ ]:
batch_size = 128
# batch_size = 4

像上面那种方法并不是一个好的选择，因为每次都要去更改源码。

第二种解决方案是在运行脚本时将`batch_size`的值作为参数传递进去。这样我们就可以根据所用的显卡来改变它。我们可以通过`sys.argv`使用命令行参数来实现这一点。

使用 swanlab.config 保存你的训练配置，例如：

超参数

输入设置，例如数据集名称或模型类型

实验的任何其他变量

swanlab.config 使你可以轻松分析你的实验并在将来复现你的工作。你还可以在SwanLab应用中比较不同实验的配置，并查看不同的训练配置如何影响模型输出。

## 2.设置实验配置
config 通常在训练脚本的开头定义。当然，不同的人工智能工作流可能会有所不同，因此 config 也支持在脚本的不同位置定义，以满足灵活的需求。

以下部分概述了定义实验配置的不同场景。

### 2.1在init中设置

下面的代码片段演示了如何使用Python字典定义 config，以及如何在初始化SwanLab实验时将该字典作为参数传递：

In [ ]:
import swanlab
swanlab.login()
# 定义一个config字典
config = {
  "hidden_layer_sizes": [64, 128],
  "activation": "ELU",
  "dropout": 0.5,
  "num_classes": 10,
  "optimizer": "Adam",
  "batch_normalization": True,
  "seq_length": 100,
}

# 在你初始化SwanLab时传递config字典
run = swanlab.init(project="config_example", config=config)

swanlab: Tracking run with swanlab version 0.3.23
swanlab: Run data will be saved locally in /content/swanlog/run-20241027_170252-eda2be8e
swanlab: 👋 Hi wenglean, welcome to swanlab!
swanlab: Syncing run ox-3 to the cloud
swanlab: 🌟 Run `swanlab watch /content/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@wenglean/config_example
swanlab: 🚀 View run at https://swanlab.cn/@wenglean/config_example/runs/21a365zn8izgx0caxk8d6


访问 config 中的值与在Python中访问其他字典的方式类似：

1. 用键名作为索引访问值

In [ ]:
hidden_layer_sizes = swanlab.config["hidden_layer_sizes"]
hidden_layer_sizes

[64, 128]

用 get() 方法访问值

In [ ]:
activation = swanlab.config.get("activation")
activation

'ELU'

用点号访问值

In [ ]:
dropout = swanlab.config.dropout
dropout

0.5